In [18]:
import pandas as pd
from plotly import express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [19]:
destinations = ['ATL', 'CHI', 'DEN', 'DFW', 'HNL', 'JFK', 'SFO']

flights = pd.DataFrame()
for dest in destinations:
    
    flights = pd.concat([flights, pd.read_csv(f'flight_LAX_{dest}_data_0601_0831.csv')])
    
airports = pd.read_csv('airport-codes_csv.csv')

In [20]:
flights

,Price,Company Name,Stops,Duration,Destination,From,Date
0,254,American Airlines,nonstop,4h 38m,ATL,LAX,6/1/23
1,73,Spirit Airlines,1 stop,25h 28m,ATL,LAX,6/1/23
2,209,American Airlines,1 stop,6h 15m,ATL,LAX,6/1/23
3,159,United Airlines,1 stop,6h 55m,ATL,LAX,6/1/23
4,204,United Airlines,1 stop,6h 10m,ATL,LAX,6/1/23
...,...,...,...,...,...,...,...
20494,982,American Airlines,1 stop,21h 55m,SFO,LAX,8/31/23
20495,712,"Spirit Airlines, Sun Country Air",2 stops,31h 15m,SFO,LAX,8/31/23
20496,702,"Spirit Airlines, Sun Country Air",2 stops,32h 27m,SFO,LAX,8/31/23
20497,737,"Spirit Airlines, Sun Country Air",2 stops,32h 14m,SFO,LAX,8/31/23


In [21]:
destinations = set(flights['Destination'])
destinations.add('LAX')

airports = airports[['name', 'iata_code', 'coordinates']]
airports = airports.rename({'name' : 'Airport', 
                            'iata_code' : 'Destination'},
                           axis = 1)

In [22]:
airports = airports[airports['Destination'].isin(destinations)]

In [23]:
airports

,Airport,Destination,coordinates
27523,Hartsfield Jackson Atlanta International Airport,ATL,"-84.428101, 33.6367"
27952,Denver International Airport,DEN,"-104.672996521, 39.861698150635"
27957,Dallas Fort Worth International Airport,DFW,"-97.038002, 32.896801"
29073,John F Kennedy International Airport,JFK,"-73.7789, 40.639801"
29188,Los Angeles International Airport,LAX,"-118.407997, 33.942501"
29785,Chicago O'Hare International Airport,ORD,"-87.9048, 41.9786"
31332,San Francisco International Airport,SFO,"-122.375, 37.61899948120117"
40449,Daniel K Inouye International Airport,HNL,"-157.924228, 21.32062"


In [24]:
airports['longitude'] = airports['coordinates'].str.split(', ').str[0].astype(float)
airports['latitude'] = airports['coordinates'].str.split(', ').str[1].astype(float)

In [25]:
airports

,Airport,Destination,coordinates,longitude,latitude
27523,Hartsfield Jackson Atlanta International Airport,ATL,"-84.428101, 33.6367",-84.428101,33.636700
27952,Denver International Airport,DEN,"-104.672996521, 39.861698150635",-104.672997,39.861698
27957,Dallas Fort Worth International Airport,DFW,"-97.038002, 32.896801",-97.038002,32.896801
29073,John F Kennedy International Airport,JFK,"-73.7789, 40.639801",-73.778900,40.639801
29188,Los Angeles International Airport,LAX,"-118.407997, 33.942501",-118.407997,33.942501
29785,Chicago O'Hare International Airport,ORD,"-87.9048, 41.9786",-87.904800,41.978600
31332,San Francisco International Airport,SFO,"-122.375, 37.61899948120117",-122.375000,37.618999
40449,Daniel K Inouye International Airport,HNL,"-157.924228, 21.32062",-157.924228,21.320620


In [26]:
LAX_lon = airports[airports['Destination'] == 'LAX']['longitude'].iloc[0]
LAX_lat = airports[airports['Destination'] == 'LAX']['latitude'].iloc[0]

airports = airports[airports['Destination'] != 'LAX']

In [27]:
flights.groupby('Destination')['Price'].aggregate([np.std, np.mean])

,std,mean
Destination,,
ATL,198.792320,430.719912
DEN,144.175585,345.503717
DFW,143.797519,302.896042
HNL,329.260149,545.536184
JFK,294.188653,494.337262
ORD,166.402091,417.926679
SFO,173.866652,410.857652


In [193]:
def visualize_airports(airports, flights, metric):
    
    lons = []
    lats = []

    for lon in airports['longitude']:

        lons.append(lon)
        lons.append(LAX_lon)

    for lat in airports['latitude']:

        lats.append(lat)
        lats.append(LAX_lat)
        
    colors = flights.groupby('Destination')['Price'].aggregate([metric])
    metric_name = colors.columns[0]
    colors = list(colors[metric_name])
    
    fig1 = px.scatter_mapbox(airports,
                             lat = 'latitude',
                             lon = 'longitude',
                             color = colors,
                             hover_data = {'latitude' : False,
                                           'longitude' : False},
                             labels = {'color' : metric_name},
                             hover_name = 'Airport')
    
    fig2 = px.scatter_mapbox(lon = [LAX_lon],
                             lat = [LAX_lat],
                             hover_name = ['Los Angeles International Airport']).add_traces(fig1.data)
    
    fig3 = px.line_mapbox(airports,
                          lat = lats,
                          lon = lons,
                          color_discrete_sequence = ['red'],
                          zoom = 2,
                          mapbox_style = 'carto-positron').add_traces(fig2.data)
    
    fig3.update_traces(marker={'size' : 20,
                               'opacity' : .5})

    fig3.show()

In [194]:
visualize_airports(airports, flights, metric = np.std)

In [195]:
visualize_airports(airports, flights, metric = np.mean)